In [1]:
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import pickle as pkl
from scipy import sparse
import seaborn as sns
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import nltk
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/mbti_1.csv")
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [3]:
types = np.unique(np.array(df['type']))
types

array(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP',
       'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'],
      dtype=object)

In [4]:
total = df.groupby(['type']).count()*50
total

,posts
type,
ENFJ,9500
ENFP,33750
ENTJ,11550
ENTP,34250
ESFJ,2100
ESFP,2400
ESTJ,1950
ESTP,4450
INFJ,73500


In [5]:
def data_clean(df, remove_special=True):
    texts = df['posts'].copy()
    labels = df['type'].copy()
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\.', ' EOSTokenDot ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\?', ' EOSTokenQuest ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'!', ' EOSTokenExs ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[\.+]', ".",x))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^\w\s]','',x))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))
    df["posts"] = df["posts"].apply(lambda x: x.lower())
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{0,3})?\b','',x)) 
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{30,1000})?\b','',x)) 
    if remove_special:
        pers_types = ['INFP' ,'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP' ,'ISFP' ,'ENTJ', 'ISTJ','ENFJ', 'ISFJ' ,'ESTP', 'ESFP' ,'ESFJ' ,'ESTJ']
        pers_types = [p.lower() for p in pers_types]
        p = re.compile("(" + "|".join(pers_types) + ")")
    return df

df1 = data_clean(df)

In [6]:
df1

,type,posts
0,INFJ,enfp intj moments sportscenter plays...
1,ENTP,finding lack these posts very alarming eo...
2,INTP,good course which know thats bles...
3,INTJ,dear intp enjoyed conversation other eos...
4,ENTJ,youre fired eostokendot thats another silly...
...,...,...
8670,ISFP,ixfp just because always think cats doms...
8671,ENFP,eostokendot eostokendot eostokendot this t...
8672,INTP,many questions when these things eostokendo...
8673,INFP,very conflicted right when comes wanting ...


In [7]:
enc = LabelEncoder()
df1['type of encoding'] = enc.fit_transform(df1['type'])
target = df1['type of encoding'] 

In [8]:
df1.head()

,type,posts,type of encoding
0,INFJ,enfp intj moments sportscenter plays...,8
1,ENTP,finding lack these posts very alarming eo...,3
2,INTP,good course which know thats bles...,11
3,INTJ,dear intp enjoyed conversation other eos...,10
4,ENTJ,youre fired eostokendot thats another silly...,2


In [9]:
print(stopwords.words('english')) # useless words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
vect = CountVectorizer(stop_words='english') 
train =  vect.fit_transform(df1["posts"])
train.shape

(8675, 98580)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, stratify=target, random_state=2022)
print ((X_train.shape),(y_train.shape),(X_test.shape),(y_test.shape))

(6940, 98580) (6940,) (1735, 98580) (1735,)


In [12]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
predictions = [round(value) for value in Y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 58.44%


In [13]:
data = pd.read_csv("Data/mbti_1.csv")
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [14]:
def get_types(row):
    t=row['type']
    I = 0; N = 0
    T = 0; J = 0    
    if t[0] == 'I': I = 1
    elif t[0] == 'E': I = 0
    else: print('I-E not found')        
    if t[1] == 'N': N = 1
    elif t[1] == 'S': N = 0
    else: print('N-S not found')     
    if t[2] == 'T': T = 1
    elif t[2] == 'F': T = 0
    else: print('T-F not found')       
    if t[3] == 'J': J = 1
    elif t[3] == 'P': J = 0
    else: print('J-P not found')
    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 
data = data.join(data.apply (lambda row: get_types (row),axis=1))
data.head(5)

,type,posts,IE,NS,TF,JP
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,1,1,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,0,1,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,1,1,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",1,1,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,0,1,1,1


In [15]:
lemmatiser = WordNetLemmatizer()
useless_words = stopwords.words("english")
unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP','ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
unique_type_list = [x.lower() for x in unique_type_list]

In [16]:
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    return [b_Pers[l] for l in personality]
def translate_back(personality):
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

list_personality_bin = np.array([translate_personality(p) for p in data.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

Binarize MBTI list: 
[[0 0 0 0]
 [1 0 1 1]
 [0 0 1 1]
 ...
 [0 0 1 1]
 [0 0 0 1]
 [0 0 0 1]]


In [17]:
def pre_process_text(data, remove_stop_words=True, remove_mbti_profiles=True):
    list_personality = []
    list_posts = []
    len_data = len(data)
    i=0
  
    for row in data.iterrows():
      posts = row[1].posts
      temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
      temp = re.sub("[^a-zA-Z]", " ", temp)
      temp = re.sub(' +', ' ', temp).lower()
      temp = re.sub(r'([a-z])\1{2,}[\s|\w]*', '', temp)
      if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in useless_words])
      else:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
      if remove_mbti_profiles:
            for t in unique_type_list:
                temp = temp.replace(t,"")
      type_labelized = translate_personality(row[1].type) 
      list_personality.append(type_labelized)
      list_posts.append(temp)
    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality
list_posts, list_personality  = pre_process_text(data, remove_stop_words=True, remove_mbti_profiles=True)

print("Example :")
print("\nData before-\n\n", data.posts[0])
print("\nPost after-\n\n", list_posts[0])
print("\nMBTI was-\n\n", data.type[0])
print("\nMBTI now-\n\n", list_personality[0])

Example :

Data before-

 'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/

In [18]:
cntizer = CountVectorizer(analyzer="word", 
                             max_features=1000,  
                             max_df=0.8,
                             min_df=0.1) 
print("Using CountVectorizer :")
X_cnt = cntizer.fit_transform(list_posts)
feature_names = list(enumerate(cntizer.get_feature_names()))
print("10 feature names can be seen below")
print(feature_names[0:10])
tfizer = TfidfTransformer()
print("\nUsing Tf-idf :")
print("Now the dataset size is as below")
X_tfidf =  tfizer.fit_transform(X_cnt).toarray()
print(X_tfidf.shape)

Using CountVectorizer :
10 feature names can be seen below
[(0, 'ability'), (1, 'able'), (2, 'absolutely'), (3, 'across'), (4, 'act'), (5, 'action'), (6, 'actually'), (7, 'add'), (8, 'advice'), (9, 'afraid')]

Using Tf-idf :
Now the dataset size is as below
(8675, 603)


In [19]:
personality_type = [ "IE: Introversion / Extroversion", "NS: Intuition / Sensing", 
                   "FT: Feeling / Thinking", "JP: Judging / Perceiving"  ]

for l in range(len(personality_type)):
    print(personality_type[l])

IE: Introversion / Extroversion
NS: Intuition / Sensing
FT: Feeling / Thinking
JP: Judging / Perceiving


In [20]:
X = X_tfidf

In [22]:

# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [33]:
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 77.98%
NS: Intuition / Sensing ,Accuracy: 87.72%
FT: Feeling / Thinking ,Accuracy: 73.20%
JP: Judging / Perceiving ,Accuracy: 63.34%


In [34]:
C = [0.1, 1, 10]
penalty = ['l1', 'l2']
solver=['newton-cg', 'lbfgs', 'liblinear']
grid = dict(C=C,penalty=penalty,solver=solver)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.650624 using {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.1, 'penalty': 'l1', 'solver': 'lbfgs'}
0.619645 (0.007113) with: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
0.624976 (0.008475) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.624928 (0.008493) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.624976 (0.008677) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 1, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 1, 'penalty': 'l1', 'solver': 'lbfgs'}
0.650624 (0.014263) with: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.643708 (0.012266) with: {'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.643708 (0.012266) with: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.643708 (0.012243) with: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 10

In [23]:
param = {}
param['solver'] = 'liblinear'
param['C'] = 1.0
param['penalty'] = 'l1' 
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
    model = LogisticRegression(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 78.85%
NS: Intuition / Sensing ,Accuracy: 87.55%
FT: Feeling / Thinking ,Accuracy: 73.49%
JP: Judging / Perceiving ,Accuracy: 64.32%


In [54]:
for l in range(len(personality_type)):
    
    Y = list_personality[:,l]

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

    # fit model on training data
    model = SVC(random_state = 1)
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    
    print("%s Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion Accuracy: 77.96%
NS: Intuition / Sensing Accuracy: 86.03%
FT: Feeling / Thinking Accuracy: 73.07%
JP: Judging / Perceiving Accuracy: 65.39%


In [38]:
#Tuning
C = [10, 20]
gamma = [0.1, 1]
kernel=['rbf']
grid = dict(C=C,gamma=gamma,kernel=kernel)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.622103 using {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.622103 (0.015751) with: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.619581 (0.017722) with: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.617056 (0.015182) with: {'C': 20, 'gamma': 0.1, 'kernel': 'rbf'}
0.619409 (0.017766) with: {'C': 20, 'gamma': 1, 'kernel': 'rbf'}


In [39]:
param = {}
param['kernel'] = 'rbf'
param['gamma'] = 0.1
param['C'] = 10
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
    model = SVC(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 78.44%
NS: Intuition / Sensing ,Accuracy: 87.67%
FT: Feeling / Thinking ,Accuracy: 73.43%
JP: Judging / Perceiving ,Accuracy: 62.13%


In [61]:
for l in range(len(personality_type)):
    
    Y = list_personality[:,l]

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)

    # fit model on training data
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    
    print("%s Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion Accuracy: 78.44%
NS: Intuition / Sensing Accuracy: 87.67%
FT: Feeling / Thinking Accuracy: 67.55%
JP: Judging / Perceiving Accuracy: 61.33%


In [63]:
n_estimators = [90, 100, 110]
# Number of features to consider at every split
max_features = ['auto', 'log2']
# Maximum number of levels in tree
max_depth = [range(2,5,1)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree

In [64]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [62]:
from sklearn.model_selection import RandomizedSearchCV

In [66]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=7, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [range(2, 5), None],
                                        'max_features': ['auto', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [90, 100, 110]},
                   random_state=7, verbose=2)

In [67]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': None}

In [77]:
param_grid = {
    'max_depth': [None],
    'max_features': ['auto'],
    'min_samples_leaf': [1],
    'min_samples_split': [3],
    'n_estimators': [ 120, 130, 140, 150]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [79]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None], 'max_features': ['auto'],
                         'min_samples_leaf': [1], 'min_samples_split': [3],
                         'n_estimators': [120, 130, 140, 150]},
             verbose=2)

In [80]:
grid_search.best_params_

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 130}

In [29]:

n_estimators = [130]
max_features = ['auto', 'log2']
max_depth = [range(2,5,1)]
min_samples_split = [range(1,10,1)]
min_samples_leaf = [range(1,5,1)]
criterion = ['gini', 'entropy']
grid = dict(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, criterion = criterion)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

IE: Introversion / Extroversion ,Accuracy: 78.44%
NS: Intuition / Sensing ,Accuracy: 87.67%
FT: Feeling / Thinking ,Accuracy: 66.05%
JP: Judging / Perceiving ,Accuracy: 59.60%


In [82]:
param = {}
param['n_estimators'] = 130
param['max_features'] = 'auto'
param['min_samples_split'] = 3
param['min_samples_leaf'] = 1
param['max_depth'] = 35
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
    model = RandomForestClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 78.44%
NS: Intuition / Sensing ,Accuracy: 87.67%
FT: Feeling / Thinking ,Accuracy: 67.55%
JP: Judging / Perceiving ,Accuracy: 61.67%


In [58]:
for l in range(len(personality_type)):

    Y = list_personality[:,l]

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

    # fit model on training data
    model = SGDClassifier() 
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    
    print("%s Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion Accuracy: 77.29%
NS: Intuition / Sensing Accuracy: 86.34%
FT: Feeling / Thinking Accuracy: 73.20%
JP: Judging / Perceiving Accuracy: 65.13%


In [42]:
#Tuning
loss = ["hinge", "log", "squared_hinge", "modified_huber"]
alpha = [0.0001, 0.001, 0.01]
penalty=["l2", "l1"]
grid = dict(loss=loss,alpha=alpha,penalty=penalty)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.638621 using {'alpha': 0.001, 'loss': 'modified_huber', 'penalty': 'l1'}
0.617973 (0.016395) with: {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2'}
0.627322 (0.015300) with: {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1'}
0.622044 (0.015545) with: {'alpha': 0.0001, 'loss': 'log', 'penalty': 'l2'}
0.622850 (0.016402) with: {'alpha': 0.0001, 'loss': 'log', 'penalty': 'l1'}
0.562285 (0.014541) with: {'alpha': 0.0001, 'loss': 'squared_hinge', 'penalty': 'l2'}
0.561080 (0.018939) with: {'alpha': 0.0001, 'loss': 'squared_hinge', 'penalty': 'l1'}
0.607307 (0.025626) with: {'alpha': 0.0001, 'loss': 'modified_huber', 'penalty': 'l2'}
0.611210 (0.030443) with: {'alpha': 0.0001, 'loss': 'modified_huber', 'penalty': 'l1'}
0.615680 (0.009077) with: {'alpha': 0.001, 'loss': 'hinge', 'penalty': 'l2'}
0.619811 (0.010876) with: {'alpha': 0.001, 'loss': 'hinge', 'penalty': 'l1'}
0.631051 (0.012549) with: {'alpha': 0.001, 'loss': 'log', 'penalty': 'l2'}
0.623482 (0.012965) with: {'alpha': 

In [60]:
param = {}
param['alpha'] = 0.001
param['loss'] = 'modified_huber'
param['penalty'] = 'l1'
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
    model = SGDClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 78.67%
NS: Intuition / Sensing ,Accuracy: 87.67%
FT: Feeling / Thinking ,Accuracy: 72.33%
JP: Judging / Perceiving ,Accuracy: 64.96%


In [44]:
for l in range(len(personality_type)):

    Y = list_personality[:,l]

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

    # fit model on training data
    model = KNeighborsClassifier(n_neighbors = 2)  # n_neighbors means k
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
   
    print("%s Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion Accuracy: 77.19%
NS: Intuition / Sensing Accuracy: 85.75%
FT: Feeling / Thinking Accuracy: 54.84%
JP: Judging / Perceiving Accuracy: 41.08%


In [46]:
#Tuning
leaf_size = list(range(1,4))
n_neighbors = list(range(3,6))
p=[1,2]
grid = dict(leaf_size=leaf_size,n_neighbors=n_neighbors,p=p)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.602546 using {'leaf_size': 1, 'n_neighbors': 5, 'p': 1}
0.440516 (0.081028) with: {'leaf_size': 1, 'n_neighbors': 3, 'p': 1}
0.468848 (0.067632) with: {'leaf_size': 1, 'n_neighbors': 3, 'p': 2}
0.439025 (0.081819) with: {'leaf_size': 1, 'n_neighbors': 4, 'p': 1}
0.453880 (0.070292) with: {'leaf_size': 1, 'n_neighbors': 4, 'p': 2}
0.602546 (0.002899) with: {'leaf_size': 1, 'n_neighbors': 5, 'p': 1}
0.596351 (0.012350) with: {'leaf_size': 1, 'n_neighbors': 5, 'p': 2}
0.440516 (0.081028) with: {'leaf_size': 2, 'n_neighbors': 3, 'p': 1}
0.468848 (0.067632) with: {'leaf_size': 2, 'n_neighbors': 3, 'p': 2}
0.439025 (0.081819) with: {'leaf_size': 2, 'n_neighbors': 4, 'p': 1}
0.453880 (0.070292) with: {'leaf_size': 2, 'n_neighbors': 4, 'p': 2}
0.602546 (0.002899) with: {'leaf_size': 2, 'n_neighbors': 5, 'p': 1}
0.596351 (0.012350) with: {'leaf_size': 2, 'n_neighbors': 5, 'p': 2}
0.440516 (0.081028) with: {'leaf_size': 3, 'n_neighbors': 3, 'p': 1}
0.468848 (0.067632) with: {'leaf_size':

In [47]:
param = {}
param['leaf_size'] = 1
param['n_neighbors'] = 5
param['p'] = 1
for l in range(len(personality_type)):
    Y = list_personality[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2022)
    model = KNeighborsClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("%s ,Accuracy: %.2f%%" % (personality_type[l], accuracy * 100.0))

IE: Introversion / Extroversion ,Accuracy: 78.44%
NS: Intuition / Sensing ,Accuracy: 87.67%
FT: Feeling / Thinking ,Accuracy: 52.97%
JP: Judging / Perceiving ,Accuracy: 58.79%
